In [89]:
#########################
#         import        #
#########################
import os
import csv
import datetime
import pandas as pd
from pandas_datareader import data
import matplotlib.pyplot as plt
import mplfinance as mpf
from tkinter import *
from tkinter import ttk
from tkcalendar import Calendar, DateEntry

In [90]:
#########################
#     Global params     #
#########################
ExStockID = 0 #検索した証券のIDコード
ExStockName = '' #検索した証券の名前
SellCount = 0

In [91]:
#########################
#       Stock Show      #
#########################
def Stock_Show():
    root = Toplevel()
    root.title("株価表示")

    fig1 = PhotoImage(file="img/test1.png")
    fig2 = PhotoImage(file="img/test2.png")

    canvas = Canvas(root, bg="white", height=550, width=1230)
    canvas.pack()
    canvas.create_image(550, 0, image=fig2, anchor=NW)
    canvas.create_image(0, 0, image=fig1, anchor=NW)

    root.mainloop()

In [92]:
#########################
#       Get Stock       #
#########################
# SoftBank Stock data from Stooq.com 
# reference https://seanmemo.com/234/

def Get_Stock(Ccode, date_data):
    # 企業名設定
    global ExStockName

    # 企業コード設定
    Ccode = Ccode + '.JP'

    # サイト設定
    Site = 'stooq'

    # 日にち設定
    tempdate = date_data.split('/')
    StartD = '20' + tempdate[2] + '-' + tempdate[0] + '-' + tempdate[1]
    date_data = datetime.date.today()
    EndD = str(date_data.year) + '-' + str(date_data.month) + '-' + str(date_data.day)

    # Stockデータ取得
    df = data.DataReader(Ccode, Site, StartD, EndD)
    print('Getting Stock finished')

    # Stockデータ描画(plt.ver)
    plt.figure()
    df['Close'].plot(label="Company Name")
    df['Close'].rolling(5).mean().plot(label='5', linestyle='dashed')
    df['Close'].rolling(25).mean().plot(label='25', linestyle='dashed')
    df['Close'].rolling(75).mean().plot(label='75', linestyle='dashed')
    plt.legend()
    plt.grid()
    #dfGraph = pandas.DataFrame({'[Company Name]':df['Close']})
    #dfGraph.plot(title='[Company Name]',grid=True)
    plt.savefig("img/test1.png")

    # Stockデータ描画(mpf.ver)
    df = df.sort_index()
    mpf.plot(df ,type='candle',
             figsize=(6.5,5),
             mav=(5, 25, 75), #mav：移動平均線の表示
             volume=True, style='yahoo',
             savefig = dict(fname='img/test2.png',dpi=100)
            )
    
    Stock_Show()

In [93]:
#########################
#    Stock Exoroler     #
#########################
def StockIdSet(selected):
    global ExStockID
    ExStockID = selected.split(',')[0]
    global ExStockName
    ExStockName = selected.split(',')[1]

def Stock_Exproler(keyword, style, window):
    with open("data/companylist.csv") as f:
        lines = f.readlines()

    stocks = [line.strip() for line in lines]
    stock_XXX = [line for line in stocks if keyword in line]
    if style == 'id':
        for stock in stock_XXX:
            if stock.split(',')[0] == keyword:
                stock_XXX = stock
                break
        else:
            stock_XXX = []

        var = str(stock_XXX)
        if not stock_XXX:
            print('Not Found')
        else:
            StockIdSet(var)
    
    

    if window == 'exproler' :
        root = Toplevel()
        root.title("企業検索")
        var = StringVar(value=stock_XXX)
        listbox = Listbox(root,
                        listvariable=var,
                        width=50
                        )
        listbox.grid(row=0, column=0)

        StockIndButton = ttk.Frame(root, padding=(0, 5))
        StockIndButton.grid(row=2, column=0, sticky=N)
        buttonInd= ttk.Button(
            StockIndButton,
            text='セット',
            command=lambda: [StockIdSet(listbox.get(listbox.curselection())), root.quit(), root.destroy()]
            )
        buttonInd.grid(row=0, column=1)

        root.mainloop()
    #print(stock_XXX)

In [94]:
#########################
#      Stock Index      #
#########################

def Stock_Index(Username):
    # set up
    root = Toplevel()
    root.title("stock_index")
    root.resizable(False, False)
    frameEntry = ttk.Frame(root)
    frameEntry.grid()

    # Corp Explore label
    labelCorpName = ttk.Label(
        frameEntry,
        text='CorpName'
        )
    labelCorpName.grid(row=0, column=0)

    # Corp ID label
    labelCorpid = ttk.Label(
        frameEntry,
        text='CorpID'
        )
    labelCorpid.grid(row=1, column=0)

    # Day label
    labelDayindex = ttk.Label(
        frameEntry,
        text='Start Day'
        )
    labelDayindex.grid(row=2, column=0)

    # Corp Name Entry
    Corpname = StringVar()
    Corpname_entry = ttk.Entry(
        frameEntry,
        textvariable = Corpname,
        width=20
        )
    Corpname_entry.grid(row=0, column=1)

    # Corp ID Entry
    Corpid = StringVar()
    Corpid_entry = Entry(
        frameEntry,
        textvariable = Corpid,
        width=20
        )
    Corpid_entry.grid(row=1, column=1)

    
    # Day Entry
    calender_date = Calendar(frameEntry, date_patternstr="y-mm-dd")
    calender_date.grid(row=2, column=1)

    # Button Setup
    StockExploreButton = ttk.Frame(frameEntry, padding=(0, 5))
    StockExploreButton.grid(row=0, column=2, sticky=W)
    buttonExplore= ttk.Button(
        StockExploreButton,
        text='検索',
        command=lambda: [Stock_Exproler(Corpname.get(), 'name', 'exproler'), Corpid_entry.delete(0, END), Corpid_entry.insert(0, ExStockID)]
        )
    buttonExplore.pack(side=LEFT)
    
    StockDrawButton = ttk.Frame(frameEntry, padding=(0, 5))
    StockDrawButton.grid(row=3, column=1, sticky=W)
    buttonDraw= ttk.Button(
        StockDrawButton,
        text='DRAW',
        command=lambda: Get_Stock(Corpid.get(), calender_date.get_date())
        )
    buttonDraw.pack(side=LEFT)

    labelUser = ttk.Label(
        frameEntry,
        text='User: ' + Username
        )
    labelUser.grid(row=4, column=2)
    
    root.mainloop()

In [95]:
#########################
#   Show owned shares   #
#########################

def List_add(Username, table, ID, Number):
    global ExStockName
    ExStockName = ''
    Stock_Exproler(ID, 'id', 'trade')
    date_data = datetime.date.today()
    TradeDay = str(date_data.year) + '-' + str(date_data.month) + '-' + str(date_data.day)

    if ExStockName != '':
        table.insert('', 'end', values=[ID, ExStockName, Number, TradeDay])
        CSV_add(Username, ID, ExStockName, Number, TradeDay)


def CSV_add(Username, ID, Name, Number, TradeDay):
    id = "code:" + ID
    name = "name:" + Name
    number = "number:" + Number
    day = "day:" + TradeDay
    s = [id + ',' + name + ',' + number + ',' + day + ',']
    with open('users/' + Username + '/stocklist.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(s)



def List_del(Username, table):
    selected = table.focus()
    # print(table.item(selected, 'values')[0])
    table.delete(selected)

    global SellCount
    index = int(selected.replace('I','')) - SellCount
    SellCount = SellCount + 1
    CSV_del(Username, (index - 1))


def CSV_del(Username, index):
    df = pd.read_csv('users/' + Username + '/stocklist.csv')
    # print(df)
    df = df.drop(index, axis=0)
    df.to_csv('users/' + Username + '/stocklist.csv', index=False)



def Show_Owned_Shares(Username):
    global ExStockName

    targetlist = ['code', 'name', 'number', 'day']

    #set up
    root = Toplevel()
    root.title("stock_index")
    root.resizable(False, False)

    #フレーム作成
    frametbl = ttk.Frame(root)
    frametbl.grid(row=0, column=0)
    frametrade = ttk.Frame(root)
    frametrade.grid(row=0, column=1)

    # 表をインスタンス化(行数を指定)
    tbl = ttk.Treeview(master=frametbl, height=10)

    # 列を作成
    tbl['columns'] = targetlist
    # ヘッダーの設定
    tbl['show'] = 'headings'
    tbl.heading('code'  , text='株式コード')
    tbl.heading('name' , text='株式名')
    tbl.heading('number', text='購入株数')
    tbl.heading('day', text='購入日')
    # 各列の幅を設定
    tbl.column('code', width=100, anchor='center')
    tbl.column('name', width=200, anchor='w')
    tbl.column('number', width=100, anchor='e')
    tbl.column('day', width=100, anchor='e')

    with open('users/' + Username + '/stocklist.csv') as f:
        stockdata = f.readlines()
    
    for stock in stockdata:
        if '#' not in stock:
            data = []
            for target in targetlist:
                idx = stock.find(target)
                s1 = stock[idx+len(target)+1:]
                t = ','
                idx = s1.find(t)
                s2 = s1[:idx]
                data.append(s2)
            tbl.insert('', 'end', values=data)

    # 作成した表を配置
    tbl.grid(row=0, column=1)


    ### frametrade ###
    # 購入株IDラベル
    labelStockId = ttk.Label(
        frametrade,
        text='株式コード'
        )
    labelStockId.grid(row=0, column=0)

    # 購入株IDエントリー
    StockId = StringVar()
    StockId_entry = ttk.Entry(
        frametrade,
        textvariable = StockId,
        width=20
    )
    StockId_entry.grid(row=0, column=1)

    # 購入株数ラベル
    labelStockNumber = ttk.Label(
        frametrade,
        text='購入株数'
    )
    labelStockNumber.grid(row=1, column=0)

    # 購入株数エントリー
    StockNumber = StringVar()
    StockNumber_entry = ttk.Entry(
        frametrade,
        textvariable = StockNumber,
        width=20
    )
    StockNumber_entry.grid(row=1, column=1)

    # 購入ボタン
    buttonStockBuy = ttk.Button(
        frametrade,
        text='購入',
        command=lambda: List_add(Username, tbl, StockId.get(), StockNumber.get())
        )
    buttonStockBuy.grid(row=2, column=1)

    # 売却ラベル
    labelStockSell = ttk.Label(
        frametrade,
        text='売却する株を表から指定してください'
    )
    labelStockSell.grid(row=3, column=0)
    # 売却ボタン
    buttonStockSell = ttk.Button(
        frametrade,
        text='売却',
        command=lambda: List_del(Username, tbl)
        )
    buttonStockSell.grid(row=3, column=1)

    root.mainloop()

In [96]:
#########################
#      User_Action      #
#########################

def User_Action_Window(Username):
    # set up
    root = Toplevel()
    root.title("テスト")
    root.resizable(False, False)

    labelUser = ttk.Label(
        root,
        text='User: ' + Username
        )
    labelUser.grid(row=0, column=0)

    # 株式検索遷移ボタン
    StockExploreButton = ttk.Frame(root, padding=(0, 5))
    StockExploreButton.grid(row=1, column=0, sticky=N)
    buttonExplore= ttk.Button(
        StockExploreButton,
        text='株式検索',
        command=lambda: Stock_Index(Username)
        )
    buttonExplore.pack(side=LEFT)

    # 株式売買遷移ボタン
    StockTradeButton = ttk.Frame(root, padding=(0, 5))
    StockTradeButton.grid(row=2, column=0, sticky=N)
    buttonTrade= ttk.Button(
        StockTradeButton,
        text='株式売買',
        command=lambda: Show_Owned_Shares(Username)
        )
    buttonTrade.pack(side=LEFT)

    # ログアウトボタン
    StockLogoutButton = ttk.Frame(root, padding=(0, 5))
    StockLogoutButton.grid(row=3, column=0, sticky=N)
    buttonLogout= ttk.Button(
        StockLogoutButton,
        text='ログアウト',
        command=lambda: (root.quit(), root.destroy())
        )
    buttonLogout.pack(side=LEFT)

    root.mainloop()

In [97]:
#########################
#    Subscribe_User     #
#########################
def MakeNewDir(dirpath):
    os.mkdir(dirpath)

def MakeUserinfo(dirpath, Username, Password):
    UN = ['username:' + Username]
    PW = ['password:' + Password]
    path = dirpath + '/userinfo.csv'

    with open(path, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(UN)
        writer.writerow(PW)
    f.close()

def MakeStocklist(dirpath):
    path = dirpath + '/stocklist.csv'
    with open(path, 'w') as f:
        writer = csv.writer(f)
        writer.writerow('#')
    f.close()

def NewUsernameCheck(Username):
    path = './users'
    files = os.listdir(path)

    if Username in files:
        return False
    else:
        return True
    
def NewPasswordCheck(Psw, RePsw):
    if Psw == RePsw:
        return True
    else:
        return False
    

def Subscribe_Complete_Window(Username):
    # set up
    root = Toplevel()
    root.title("新規登録ありがとうございます")
    root.resizable(False, False)

    frameMessage = ttk.Frame(root)
    frameMessage.grid(row=0, column=0, pady=15)

    frameCloseButton = ttk.Frame(root)
    frameCloseButton.grid(row=1, column=0, pady=15)

    # ユーザ名表示
    labelUsername = ttk.Label(
        frameMessage,
        text= Username + '様、',
        font=('MSゴシック',20)
        )
    labelUsername.grid(row=0, column=0)

    # メッセージ1
    labelMessage = ttk.Label(
        frameMessage,
        text='新規登録が完了しました！！',
        font=('MSゴシック',20)
        )
    labelMessage.grid(row=1, column=0)

    # メッセージ2
    labelMessage = ttk.Label(
        frameMessage,
        text='このウィンドウは閉じて構いません',
        )
    labelMessage.grid(row=2, column=0)

    # 閉じるボタン
    buttonClose = ttk.Button(
        frameCloseButton,
        text='閉じる',
        command=lambda: [root.quit(), root.destroy()]
        )
    buttonClose.pack(anchor='center')

    root.mainloop()


def Subscribe(root, Username, UsernameMessage, Password, Repassword, PasswordMessage):
    UNcheck = NewUsernameCheck(Username)
    PWcheck = NewPasswordCheck(Password, Repassword)

    if  UNcheck:
        UsernameMessage["text"] = ''
        if PWcheck:
            PasswordMessage["text"] = ''
            dirpath = './users/' + Username
            MakeNewDir(dirpath)
            MakeUserinfo(dirpath, Username, Password)
            MakeStocklist(dirpath)
            root.quit()
            root.destroy()
            Subscribe_Complete_Window(Username)
        else:
            PasswordMessage["text"] = '異なるパスワードが入力されています'
    else:
        UsernameMessage["text"] = 'このユーザ名はすでに使われています'



def Subscribe_Window():
    # set up
    root = Toplevel()
    root.title("新規登録画面")
    root.resizable(False, False)

    frameUsername = ttk.Frame(root)
    frameUsername.grid(row=0, column=0, pady=15)

    framePassword = ttk.Frame(root)
    framePassword.grid(row=1, column=0, pady=15)

    frameSubscribeButton = ttk.Frame(root)
    frameSubscribeButton.grid(row=2, column=0, pady=15, sticky=E)

    # ユーザ名ラベルタイトル
    labelUsernameTitle = ttk.Label(
        frameUsername,
        text='||ユーザ名登録',
        font=('MSゴシック',20)
        )
    labelUsernameTitle.grid(row=0, column=0)

    # ユーザ名ラベル
    labelUsername = ttk.Label(
        frameUsername,
        text='ユーザ名(半角のみ)'
        )
    labelUsername.grid(row=1, column=0)

    # ユーザ名エントリー
    Username = StringVar()
    Username_entry = ttk.Entry(
        frameUsername,
        textvariable = Username,
        width=20
    )
    Username_entry.grid(row=1, column=1)

    # ユーザ名再入力要求メッセージ
    labelUsernameMessage = ttk.Label(
        frameUsername,
        text='',
        foreground='#ff0000'
        )
    labelUsernameMessage.grid(row=2, column=1)


    # パスワードラベルタイトル
    labelPasswordTitle = ttk.Label(
        framePassword,
        text='||パスワード登録',
        font=('MSゴシック',20)
        )
    labelPasswordTitle.grid(row=0, column=0)

    # パスワードラベル
    labelPsw = ttk.Label(
        framePassword,
        text='パスワード'
        )
    labelPsw.grid(row=1, column=0)

    #パスワードエントリー
    Psw = StringVar()
    Psw_entry = ttk.Entry(
        framePassword,
        textvariable = Psw,
        width=20,
        show='*'
    )
    Psw_entry.grid(row=1, column=1)

    # パスワード(再入力)ラベル
    labelRePsw = ttk.Label(
        framePassword,
        text='パスワード(再入力)'
        )
    labelRePsw.grid(row=2, column=0)

    #パスワード(再入力)エントリー
    RePsw = StringVar()
    RePsw_entry = ttk.Entry(
        framePassword,
        textvariable = RePsw,
        width=20,
        show='*'
    )
    RePsw_entry.grid(row=2, column=1)

    # ユーザ名再入力要求メッセージ
    labelPasswordMessage = ttk.Label(
        framePassword,
        text='',
        foreground='#ff0000'
        )
    labelPasswordMessage.grid(row=3, column=1)

    # 登録ボタン
    buttonSubscribe = ttk.Button(
        frameSubscribeButton,
        text='新規登録',
        command=lambda: Subscribe(root, Username.get(), labelUsernameMessage, Psw.get(), RePsw.get(), labelPasswordMessage)
        )
    buttonSubscribe.pack(anchor='center')

In [98]:
#########################
#          login        #
#########################
def PswCheck(Username, Password):
    path = './users'
    files = os.listdir(path)

    if Username in files:
        path = path + '/' + Username
        csvs = os.listdir(path)
        if 'userinfo.csv' in csvs:
            with open(path + '/userinfo.csv') as f:
                lines = f.readlines()
            userinfos = [line.strip() for line in lines]
            for info in userinfos:
                if info.find('password') == 0:
                    idx = info.find('password')
                    correct = info[idx + 9:]
                    if correct == Password:
                        return True
                    else:
                        return False
            else:
                print('csvが破損している可能性あり')
    else:
        return False
    
def Login(check, PswEntry, message, Username):
    PswEntry.delete(0, END)
    if check:
        message["text"] = ''
        User_Action_Window(Username)
    else:
        message["text"] = 'ユーザ名またはパスワードが不正です'

def Login_Window():
    # set up
    root = Tk()
    root.title("test_login")
    root.resizable(False, False)

    # フレーム設計
    frameSubscribe = ttk.Frame(root)
    frameSubscribe.grid(row=0, column=0)

    frameLogin = ttk.Frame(root)
    frameLogin.grid(row=1, column=0, pady=15)

    frameEntryButton = ttk.Frame(root)
    frameEntryButton.grid(row=2, column=0)

    ### frameSubscribe
    # 新規登録文
    labelSubscribe = ttk.Label(
        frameSubscribe,
        text='新規ユーザ登録はこちらから→'
        )
    labelSubscribe.grid(row=0, column=0)
    buttonSubscribe = ttk.Button(
        frameSubscribe,
        text='新規登録(未実装)',
        command=lambda: Subscribe_Window()
        )
    buttonSubscribe.grid(row=0, column=1)

    ### frameLogin
    # ユーザ名ラベルタイトル
    labelUsernameTitle = ttk.Label(
        frameLogin,
        text='||ログイン',
        font=('MSゴシック',20)
        )
    labelUsernameTitle.grid(row=0, column=0)

    # ユーザ名ラベル
    labelUsername = ttk.Label(
        frameLogin,
        text='ユーザ名'
        )
    labelUsername.grid(row=1, column=0)

    # ユーザ名エントリー
    Username = StringVar()
    Username_entry = ttk.Entry(
        frameLogin,
        textvariable = Username,
        width=20
    )
    Username_entry.grid(row=1, column=1)

    # パスワードラベル
    labelPsw = ttk.Label(
        frameLogin,
        text='パスワード'
        )
    labelPsw.grid(row=2, column=0)

    #パスワードエントリー
    Psw = StringVar()
    Psw_entry = ttk.Entry(
        frameLogin,
        textvariable = Psw,
        width=20,
        show='*'
    )
    Psw_entry.grid(row=2, column=1)

    # ユーザ名orパスワード間違いメッセージ
    labelmessage = ttk.Label(
        frameLogin,
        text='',
        foreground='#ff0000'
        )
    labelmessage.grid(row=3, column=1)


    ### frameEntryButton
    # ログインボタン
    buttonCancel = ttk.Button(
        frameEntryButton,
        text='キャンセル',
        command=quit
        )
    buttonCancel.grid(row=0,column=0, padx=20)

    # キャンセルボタン
    buttonLogin = ttk.Button(
        frameEntryButton,
        text='ログイン',
        command=lambda: Login(PswCheck(Username.get(), Psw.get()), Psw_entry, labelmessage, Username.get())
        )
    buttonLogin.grid(row=0, column=1, padx=20)

    root.mainloop()

In [99]:
#########################
#          main         #
#########################

Login_Window()
#Subscribe_Complete_Window('UEC')
#User_Action_Window('test')
#Stock_Index('123')

#Get_Stock('9766', '2023-6-1')
#Stock_Exproler('A')

#Show_Owned_Shares('test')
#Stock_Exproler('2593','id','trade')